In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#カレントディレクトリに移動
%cd "/content/drive/My Drive/Google Colab"

/content/drive/My Drive/Google Colab


# 以下没コード！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！

In [3]:
# モデルの構築
import os
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

#フォルダをクラス名にする
path = "open_image"
folders = os.listdir(path)

#フォルダ名を抽出
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
n_classes = len(classes)

#画像とラベルの格納
X = []
Y = []

#画像を読み込みリサイズする
for label,class_name in enumerate(classes):
  files = glob.glob(path + "/" +  class_name + "/*.png")

  for file in files:
    img = cv2.imread(file)
    print(file)
    img = cv2.resize(img,dsize=(64,64))
    X.append(img)
    Y.append(label)

#精度を上げるために正規化
X = np.array(X)
X = X.astype('float32')
X /= 255.0

#ラベルの変換

Y = np.array(Y)
Y = np_utils.to_categorical(Y,n_classes)
Y[:5]

#学習データとテストデータに分ける(テストデータ2割、学習データ8割)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)


#学習データ(8割)
print(X_train.shape)
#テストデータ(2割)
print(X_test.shape)
#学習データ(8割)
print(Y_train.shape)
#テストデータ(2割)
print(Y_test.shape)

print(img)

open_image/03/m_204_03.png
open_image/03/m_205_03.png
open_image/03/m_220_03.png
open_image/03/m_221_03.png
open_image/03/m_237_03.png
open_image/03/m_238_03.png
open_image/03/m_256_03.png
open_image/03/m_257_03.png
open_image/03/m_258_03.png
open_image/03/m_259_03.png
open_image/03/m_279_03.png
open_image/03/m_280_03.png
open_image/03/m_295_03.png
open_image/03/m_296_03.png
open_image/03/m_297_03.png
open_image/03/03-01-01-01-01-01-04.png
open_image/03/03-01-03-01-01-02-04.png
open_image/03/03-01-03-02-02-01-04.png
open_image/03/03-01-03-01-01-01-04.png
open_image/03/03-01-03-02-02-02-04.png
open_image/03/03-01-03-02-01-01-04.png
open_image/03/03-01-03-02-01-02-04.png
open_image/03/03-01-03-01-02-01-04.png
open_image/03/03-01-03-01-02-02-04.png
open_image/03/03-01-01-01-01-01-05.png
open_image/03/03-01-03-01-01-02-05.png
open_image/03/03-01-03-01-02-02-05.png
open_image/03/03-01-03-01-02-01-05.png
open_image/03/03-01-03-01-01-01-05.png
open_image/03/03-01-03-02-01-01-05.png
open_image

In [ ]:
!pip install keras-layer-normalization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=d898a4b8e7c61b6dcd78efd119b377e114cc5657c08d6418b3e032cafff65fc3
  Stored in directory: /root/.cache/pip/wheels/d7/2b/f4/28f4bab995fa99c26b761bc7f9aeb5bf6c81e9be6ccd0b853b
Successfully built keras-layer-normalization


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Activation, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

#vgg16
input_tensor = Input(shape=(64,64,3))
#最後の1000の層を省く
base_model = VGG16(weights='imagenet', input_tensor=input_tensor,include_top=False)


#後付けで入れたい層の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation='softmax'))


#結合
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))


#学習させない層
for layer in model.layers[:15]:
  layer.trainable = False

print('# layers=', len(model.layers))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

# layers= 20
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 12

In [ ]:
#学習データで学習
model.fit(X_train, Y_train, epochs=20, batch_size=16)

#テストデータで精度確認
# score = model.evaluate(X_test, Y_test, batch_size=16)

Epoch 1/20
69/69 [==============================] - 3s 19ms/step - loss: 2.1321 - accuracy: 0.1332
Epoch 2/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0765 - accuracy: 0.1395
Epoch 3/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0744 - accuracy: 0.1395
Epoch 4/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0726 - accuracy: 0.1395
Epoch 5/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0712 - accuracy: 0.1395
Epoch 6/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0698 - accuracy: 0.1395
Epoch 7/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0686 - accuracy: 0.1395
Epoch 8/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0678 - accuracy: 0.1395
Epoch 9/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0670 - accuracy: 0.1395
Epoch 10/20
69/69 [==============================] - 1s 18ms/step - loss: 2.0664 - accuracy: 0.1395
Epoch 11/

In [ ]:
from keras.models import load_model
import pickle
import cv2
model = load_model('cnn.h5')
print(Y_test)

# testの精度を出す！

[[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [ ]:

data = model.predict(X_test)
a = []
b = []
for i in data:
  a.append(np.argmax(i))
for j in Y_test:
  b.append(np.argmax(j))
print(a)
print(b)
a = np.array(a)
b = np.array(b)
c = a == b
count = 0
for k in c:
  if k == True:
    count+=1
print(count/len(c))

9/9 [==============================] - 0s 20ms/step
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[7, 3, 1, 3, 2, 0, 7, 4, 7, 4, 1, 3, 4, 7, 5, 7, 2, 1, 4, 0, 4, 3, 4, 5, 3, 4, 7, 3, 6, 4, 6, 2, 3, 4, 2, 7, 1, 2, 3, 6

In [ ]:
from keras.models import load_model
import pickle
import cv2

#モデルとクラス名の読み込み
model = load_model('cnn.h5')
classes = pickle.load(open('classes.sav', 'rb'))

#sample画像の前処理
img = cv2.imread('m_110_01.png')
img = cv2.resize(img,dsize=(64,64))
img = img.astype('float32')
img /= 255.0
img = img[None, ...]

result = model.predict(img)

#確率が一番大きいクラス
pred = result.argmax()

print(pred)

img = cv2.imread(file)
cv2.imwrite('output/' + str(classes[pred]) + '/' + file,img)


error: ignored

In [ ]:
import pickle
#クラス名の保存
pickle.dump(classes, open('classes2.sav', 'wb'))
#モデルの保存
model.save('cnn2.h5')

In [ ]:
# ちっちゃいデータのモデル
from keras.models import load_model
import pickle
import cv2
model = load_model('cnn.h5')
classes = pickle.load(open('classes.sav', 'rb'))

data = model.predict(X_test)
a = []
b = []
for i in data:
  a.append(np.argmax(i))
for j in Y_test:
  b.append(np.argmax(j))
print(a)
print(b)
a = np.array(a)
b = np.array(b)
c = a == b
count = 0
for k in c:
  if k == True:
    count+=1
print(count/len(c))

1/1 [==============================] - 0s 156ms/step
[2, 4, 1, 5, 1, 0, 0, 7, 6, 2, 1, 2, 6, 1, 5, 2, 2, 1, 2, 0, 0]
[3, 2, 3, 5, 1, 0, 0, 4, 5, 2, 1, 1, 3, 1, 5, 5, 6, 7, 1, 0, 0]
0.47619047619047616


# ここまで没コード<br>
============================================================================
# 以下OKコード

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop # TensorFlow1系
# from keras.optimizers import RMSprop # エラー（ImportError: cannot import name 'RMSprop' from 'keras.optimizers' (/usr/local/lib/python3.7/dist-packages/keras/optimizers.py)）が発生
# from tensorflow.keras.optimizers import RMSprop # TensorFlow2系
import tensorflow as tf
import tensorflow.keras as keras

from keras.utils import np_utils
import keras
import numpy as np

num_classes = len(classes)
image_size = 64

"""
モデルを学習する関数
"""
def train(X, y, X_test, y_test):
    params = {
    'horizontal_flip': True,
    'height_shift_range': 0.1,
    'width_shift_range': 0.1,
    }
    generator = keras.preprocessing.image.ImageDataGenerator(**params)

    train_iter = generator.flow(x=X, y=y)
    model = Sequential()

    # Xは(1200, 64, 64, 3)
    # X.shape[1:]とすることで、(64, 64, 3)となり、入力にすることが可能です。
    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(8)) # 8感情に分ける
    model.add(Activation('softmax'))

    # https://keras.io/ja/optimizers/
    # 今回は、最適化アルゴリズムにRMSpropを利用
    opt = RMSprop(lr=0.00005, decay=1e-6)
    # https://keras.io/ja/models/sequential/
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X,y, batch_size=28, epochs=300)
    # HDF5ファイルにKerasのモデルを保存
    model.save('./cnn_no_200.h5')

    return model

"""
メイン関数
データの読み込みとモデルの学習を行います。
"""
def main():
    # データの読み込み
    # X_train, y_train, X_test, y_test = X_train,Y_train,X_test,Y_test

    # モデルの学習
    model = train(X_train, Y_train, X_test, Y_test)

main()

Epoch 1/300


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


20/20 [==============================] - 9s 22ms/step - loss: 2.0567 - accuracy: 0.1452
Epoch 2/300
20/20 [==============================] - 0s 12ms/step - loss: 2.0144 - accuracy: 0.1618
Epoch 3/300
20/20 [==============================] - 0s 13ms/step - loss: 2.0012 - accuracy: 0.1673
Epoch 4/300
20/20 [==============================] - 0s 12ms/step - loss: 1.9782 - accuracy: 0.2004
Epoch 5/300
20/20 [==============================] - 0s 12ms/step - loss: 1.9679 - accuracy: 0.1857
Epoch 6/300
20/20 [==============================] - 0s 12ms/step - loss: 1.9565 - accuracy: 0.2114
Epoch 7/300
20/20 [==============================] - 0s 12ms/step - loss: 1.9484 - accuracy: 0.2188
Epoch 8/300
20/20 [==============================] - 0s 12ms/step - loss: 1.9218 - accuracy: 0.1838
Epoch 9/300
20/20 [==============================] - 0s 13ms/step - loss: 1.9268 - accuracy: 0.2132
Epoch 10/300
20/20 [==============================] - 0s 12ms/step - loss: 1.8958 - accuracy: 0.2408
Epoch 11/30

In [8]:
from keras.models import load_model
import pickle
import cv2
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

model = load_model('cnn_no_200.h5')
# classes = pickle.load(open('classes.sav', 'rb'))

data = model.predict(X_test)
for k in range(len(data)):
  a_argsorted = np.argsort(data[k], axis=0)
  print(a_argsorted)
  

a = []
b = []
for i in data:
  a.append(np.argmax(i))
for j in Y_test:
  b.append(np.argmax(j))
print(b)
print(a)
a = np.array(a)
b = np.array(b)
c = a == b
count = 0
for k in c:
  if k == True:
    count+=1
print(count/len(c))
print(accuracy_score(a, b))
print(confusion_matrix(a, b))


5/5 [==============================] - 0s 6ms/step
[7 1 5 2 3 0 4 6]
[3 1 7 0 6 2 5 4]
[3 1 4 6 0 7 5 2]
[6 4 1 5 2 3 7 0]
[1 5 3 6 4 0 7 2]
[7 0 1 4 3 6 2 5]
[7 4 1 2 0 6 3 5]
[6 5 4 2 7 3 0 1]
[1 3 6 0 2 5 7 4]
[2 7 3 4 6 5 1 0]
[7 3 5 2 1 0 6 4]
[1 7 4 5 6 2 3 0]
[5 7 3 2 1 6 0 4]
[3 1 0 7 5 2 6 4]
[7 3 1 0 2 4 6 5]
[2 7 4 0 1 6 5 3]
[6 3 1 5 0 4 7 2]
[1 7 6 3 4 2 5 0]
[1 7 2 5 6 4 0 3]
[4 6 0 1 5 3 7 2]
[2 7 1 6 5 0 4 3]
[1 2 7 5 3 6 0 4]
[1 5 3 6 0 7 2 4]
[1 4 3 6 7 2 0 5]
[1 4 7 0 2 5 6 3]
[1 4 5 7 2 3 6 0]
[7 6 1 3 4 2 5 0]
[7 1 5 2 0 6 4 3]
[1 3 2 4 7 5 0 6]
[1 2 7 4 0 3 5 6]
[7 3 1 5 2 4 6 0]
[7 5 1 3 6 4 0 2]
[1 4 7 3 0 6 2 5]
[6 1 5 2 4 7 0 3]
[6 7 1 5 0 3 2 4]
[3 5 1 7 0 6 4 2]
[1 7 5 2 3 6 0 4]
[2 1 7 6 4 5 0 3]
[1 7 4 3 6 2 0 5]
[2 1 3 0 5 4 7 6]
[7 1 0 2 4 3 6 5]
[1 7 5 2 6 4 0 3]
[4 2 1 5 7 6 3 0]
[2 7 1 4 5 6 3 0]
[5 1 4 6 3 2 0 7]
[1 3 6 7 0 4 2 5]
[4 2 1 0 3 6 5 7]
[7 1 3 5 2 6 0 4]
[1 3 4 5 6 0 7 2]
[1 0 7 3 5 4 2 6]
[1 4 7 2 0 6 3 5]
[1 2 4 7 5 6 3 0]
[5 1 7 0 4 3 

### 実データで予測

In [ ]:
#フォルダをクラス名にする
path = "image"
folders = os.listdir(path)

#フォルダ名を抽出
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
n_classes = len(classes)

#画像とラベルの格納
X_pred = []
Y_pred = []

#画像を読み込みリサイズする
for label,class_name in enumerate(classes):
  files = glob.glob(path + "/" +  class_name + "/*.png")
  for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img,dsize=(64,64))
    X_pred.append(img)
    Y_pred.append(label)

#精度を上げるために正規化
X_pred = np.array(X_pred)
X_pred = X_pred.astype('float32')
X_pred /= 255.0

#ラベルの変換

Y_pred = np.array(Y_pred)
Y_pred = np_utils.to_categorical(Y_pred,n_classes)
Y_pred[:5]

#学習データとテストデータに分ける(テストデータ2割、学習データ8割)
X_train1,X_test1,Y_train1,Y_test1 = train_test_split(X_pred,Y_pred,test_size=0.2)
#学習データ(8割)
print(X_train1.shape)
#テストデータ(2割)
print(X_test1.shape)
#学習データ(8割)
print(Y_train1.shape)
#テストデータ(2割)
print(Y_test1.shape)


NameError: ignored

In [ ]:
model = load_model('cnn_open_400.h5')
# classes = pickle.load(open('classes.sav', 'rb'))

data = model.predict(X_test1)
print()
a = []
b = []
for i in data:
  a.append(np.argmax(i))
for j in Y_test1:
  b.append(np.argmax(j))
print(a)
print(b)
a = np.array(a)
b = np.array(b)
c = a == b
count = 0
for k in c:
  if k == True:
    count+=1
print(count/len(c))

1/1 [==============================] - 0s 73ms/step
[3, 3, 6, 4, 7, 4, 2, 3, 3, 6, 3, 5, 6, 7, 2, 7, 4, 6, 6, 3, 6]
[3, 1, 0, 5, 6, 0, 7, 3, 5, 4, 2, 2, 6, 3, 6, 2, 1, 7, 7, 6, 0]
0.14285714285714285


In [ ]:
model = load_model('cnn_200.h5')
# classes = pickle.load(open('classes.sav', 'rb'))

data = model.predict(X_test)
a = []
b = []
for i in data:
  a.append(np.argmax(i))
for j in Y_test:
  b.append(np.argmax(j))
print(a)
print(b)
a = np.array(a)
b = np.array(b)
c = a == b
count = 0
for k in c:
  if k == True:
    count+=1
print(count/len(c))

NameError: ignored

In [ ]:
import os
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import load_model
import pickle


model = load_model('cnn_200.h5')
img = cv2.imread("./output/myrecording101.png")
img = cv2.resize(img,dsize=(64,64))
img = img.astype('float32')
img /= 255.0
img = img[None, ...]

result = model.predict(img)
result = np.array(result)

np.argmax(result)

1/1 [==============================] - 7s 7s/step


7

In [ ]:
!pip install sounddevice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import librosa as lb
import librosa.display
import matplotlib.pyplot as plt
import numpy as np


def voice_input():
   
    duration = 3  # 3秒間録音する
    # i = i+100

    # デバイス情報関連
    sd.default.device = [1, 3] # Input, Outputデバイス指定
    input_device_info = sd.query_devices(device=sd.default.device[1])
    sr_in = int(input_device_info["default_samplerate"])

    # 録音
    myrecording = sd.rec(int(duration * sr_in), samplerate=sr_in, channels=2)
    sd.wait() # 録音終了待ち

    print(myrecording.shape) #=> (duration * sr_in, channels)

    # 録音信号のNumPy配列をwav形式で保存
    sf.write("G:/マイドライブ/Google Colab/wav/myrecording.wav", myrecording, sr_in)
  
def png_trans():
  y,sr = lb.load(f'G:/マイドライブ/Google Colab/wav/myrecording.wav') # 音声読み込み
  S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, win_length=512, hop_length=512) # メルスペクトログラムに変換
  S_dB = librosa.power_to_db(S, ref=np.max)

  # プロット
  img = librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='linear') #yの値はlinearにしないとおかしくなった
  # filename = filename.replace('.wav','')
  # filename = filename.replace('C:/Users/20t311/Documents/大学関連/実験2/myvoice/angry/','')
  # filename = f'{filename}'

  plt.savefig("G:/マイドライブ/Google Colab/output/myrecording301.png", format="png", dpi=300)

OSError: ignored

In [ ]:
img = cv2.imread("output/myrecording0.png")
print(img)
img = cv2.resize(img,dsize=(64,64))
img = img.astype('float32')
img /= 255.0
img = img[None, ...]

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
